In [47]:
import os

os.makedirs("docs", exist_ok=True)

print("docs/ 目录已准备好。请把所有 .md 文件放在 docs/ 下面（至少要有 index.md）。")
print("当前 docs 目录下的文件：", os.listdir("docs"))


OSError: [WinError 433] A device which does not exist was specified: 'docs'

In [ ]:
import os
import shutil
import re

SRC_DIR = "docs"       # 你的源 md 文件
OUT_DIR = "gen_docs"   # 自动生成 + 自动加链接的 md，用于 mkdocs

# ================================
# 0. 检查并准备目录
# ================================
if not os.path.isdir(SRC_DIR):
    raise FileNotFoundError("请先创建 docs/ 目录并放入 index.md 等源文件。")

os.makedirs(OUT_DIR, exist_ok=True)

# 清空 gen_docs
for f in os.listdir(OUT_DIR):
    path = os.path.join(OUT_DIR, f)
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

print("gen_docs/ 已清空并准备好。")

# ================================
# 1. 拷贝 md 源文件 → gen_docs
# ================================
md_files = [f for f in os.listdir(SRC_DIR) if f.lower().endswith(".md")]
md_files.sort()

if "index.md" not in [f.lower() for f in md_files]:
    raise FileNotFoundError("docs 中必须包含 index.md 作为首页。")

for f in md_files:
    shutil.copy(os.path.join(SRC_DIR, f), os.path.join(OUT_DIR, f))

print("源 md 文件已复制到 gen_docs/。")

# ================================
# 2. 读取标题
# ================================
titles = {}

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    title = None
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("#"):
                title = line.strip().lstrip("#").strip()
                break
    if not title:
        title = os.path.splitext(fname)[0]
    titles[fname] = title

print("已读取页面标题。")

# ================================
# 3. 自动生成 mkdocs.yml
# ================================
yml = []
yml.append("site_name: Jenny Website")
yml.append("use_directory_urls: false")  # 链接变成 xxx.html，不用目录形式
yml.append("docs_dir: gen_docs")        # 关键：告诉 MkDocs 使用 gen_docs 里的 md
yml.append("nav:")

for fname in md_files:
    yml.append(f"  - {titles[fname]}: {fname}")

yml.append("theme:")
yml.append("  name: material")

with open("mkdocs.yml", "w", encoding="utf-8") as f:
    f.write("\n".join(yml))

print("mkdocs.yml 已生成。")

# ================================
# 4. 自动给 gen_docs/*.md 添加链接
# ================================
AUTO_START = "<!-- AUTO-LINKS-START -->"
AUTO_END = "<!-- AUTO-LINKS-END -->"

pattern = re.compile(r"\n?\s*<!-- AUTO-LINKS-START -->.*?<!-- AUTO-LINKS-END -->", re.S)

for fname in md_files:
    path = os.path.join(OUT_DIR, fname)
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # 删除旧自动区块
    content = pattern.sub("", content).rstrip() + "\n\n"

    if fname.lower() == "index.md":
        others = [(titles[f], f) for f in md_files if f.lower() != "index.md"]
        links = "\n".join(f"- [{t}]({f})" for t, f in others) or "（暂无子页面）"

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "## 页面导航\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )
    else:
        others = [(titles[f], f) for f in md_files if f != fname]
        links = "\n".join(f"- [{t}]({f})" for t, f in others)

        auto = (
            f"{AUTO_START}\n\n"
            "---\n\n"
            "[返回首页](index.md)\n\n"
            "## 更多页面\n\n"
            f"{links}\n\n"
            f"{AUTO_END}\n"
        )

    with open(path, "w", encoding="utf-8") as f:
        f.write(content + auto)

print("gen_docs/ 中所有 md 已自动插入链接。")

print("\n✨ 已完成：复制、生成链接、生成 mkdocs.yml。接下来运行 mkdocs build。")


gen_docs/ 已清空并准备好。
源 md 文件已复制到 gen_docs/。
已读取页面标题。
mkdocs.yml 已生成。
gen_docs/ 中所有 md 已自动插入链接。

✨ 已完成：复制、生成链接、生成 mkdocs.yml。接下来运行 mkdocs build。


In [ ]:
import sys, subprocess
result = subprocess.run(
    [sys.executable, "-m", "mkdocs", "build"],
    capture_output=True,
    text=True
)
print("stdout:\n", result.stdout)
print("stderr:\n", result.stderr)


stdout:
 
stderr:
 INFO    -  Cleaning site directory
INFO    -  Building documentation to directory: g:\My Drive\Github_chan\chan\site
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\rong5\AppData\Roaming\Python\Python312\site-packages\mkdocs\__main__.py", line 370, in <module>
    cli()
  File "c:\ProgramData\anaconda3\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\click\core.py", line 1078, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\click\core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\click\core.py", line 1434, in invoke
  

In [ ]:
import os
import shutil

src = "site"
dst = "."

for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)

    # 删除旧文件/文件夹
    if os.path.isdir(d):
        shutil.rmtree(d)
    elif os.path.exists(d):
        os.remove(d)

    # 复制新的
    if os.path.isdir(s):
        shutil.copytree(s, d)
    else:
        shutil.copy2(s, d)

print("网站文件已复制到根目录。")


PermissionError: [WinError 5] Access is denied: '.\\assets\\javascripts\\lunr\\min'